In [8]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [9]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
name = 'stide_mode_flag_v2'
path = f'../persistent_data/{name}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,correct_alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,f1_cfa,scenario,ngram,window,algorithm,detection_time
0,18124,375,1230587,176757,172,174,0.988506,4,0,0.988506,0.977273,0.314442,0.982857,CVE-2017-7529/,3,100,stide,1.550473
1,62580,1223,1229739,132301,172,174,0.988506,4,0,0.988506,0.977273,0.123297,0.982857,CVE-2017-7529/,3,1000,stide,1.558355
2,17954,387,1228761,176927,172,174,0.988506,5,0,0.988506,0.971751,0.307692,0.980057,CVE-2017-7529/,5,100,stide,1.653479
3,62238,1235,1227913,132643,172,174,0.988506,5,0,0.988506,0.971751,0.122246,0.980057,CVE-2017-7529/,5,1000,stide,1.633279
4,17612,380,1226954,177269,172,174,0.988506,5,0,0.988506,0.971751,0.311594,0.980057,CVE-2017-7529/,7,100,stide,1.767648
5,111,1090,2848882,286118,2,100,0.020000,16,0,0.020000,0.111111,0.001832,0.033898,CVE-2014-0160/,3,100,stide,3.390648
6,0,2259,2847713,286229,0,100,0.000000,5,0,0.000000,0.000000,0.000000,0.000000,CVE-2014-0160/,3,1000,stide,3.361521
7,61898,1230,1226104,132983,172,174,0.988506,5,0,0.988506,0.971751,0.122682,0.980057,CVE-2017-7529/,7,1000,stide,1.880559
8,0,732,2847540,286229,0,100,0.000000,7,0,0.000000,0.000000,0.000000,0.000000,CVE-2014-0160/,5,100,stide,3.526395
9,0,2320,2845952,286229,0,100,0.000000,4,0,0.000000,0.000000,0.000000,0.000000,CVE-2014-0160/,5,1000,stide,3.701172


In [14]:
conn = sql.connect('stide_mode_flag.db')
try:
    result_list.to_sql(name, conn)
except ValueError:
    print('Table already exists')

In [25]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])
name = name[0]

stide_mode_flag_v2


In [40]:
result = pd.read_sql(f'SELECT ngram, \
                              window, \
                              AVG(consecutive_false_positives_normal) as CFP_normal, \
                              AVG(consecutive_false_positives_exploits) as CFP_exploit, \
                              AVG(detection_rate) as DR, \
                              detection_time as time \
                              FROM  {name} \
                     GROUP BY ngram, window', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,ngram,window,CFP_normal,CFP_exploit,DR,time
0,3,100,10.5,0.9,0.693332,1.550473
1,3,1000,7.9,0.3,0.880905,1.558355
2,5,100,14.2,0.1,0.681474,1.653479
3,5,1000,11.4,0.3,0.647666,1.633279
4,7,100,11.2,0.1,0.619181,1.767648
5,7,1000,11.3,0.5,0.634506,1.880559
